In [ ]:
# imports
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import fiona
from shapely.geometry import Point, Polygon, MultiPolygon
import geopandas as gpd
import pandas as pd
from pyproj import Proj, transform
ox.config(use_cache=True, log_console=True)
ox.__version__

### let's get polygons of some certain areas and add them to a dataframe

In [ ]:
place_names = ['Danubyu, Ayeyarwady Region, Myanmar', 
               'Gyobingauk, Bago Region, Myanmar',
               'Katha, Shan Region, Myanmar']
# https://www.citypopulation.de/Myanmar-Cities.html

In [ ]:
small_cities = ox.gdf_from_places(place_names, gdf_name='small_cities')
small_cities

### Evaluating if a certain point is inside a polygon

In [ ]:
# Shwe Yaung Pya Sedi
small_cities["geometry"][0].contains(Point(95.585605, 17.271707))
# anecdote time :-)

In [ ]:
# project the geometry to the appropriate UTM zone then plot it
small_cities = ox.project_gdf(small_cities)
fig, ax = ox.plot_shape(small_cities)

In [ ]:
# danubyu
small_cities["geometry"][0]

In [ ]:
print(small_cities["geometry"][0])

In [ ]:
### loading cities from an external csv file 
df = pd.read_csv('data/vectors/cities5k.csv', encoding="utf-8", sep=",", header=None, low_memory=False)
df.head()

In [ ]:
# rename the columns
df2 = df[[1,4,5,14]]
df2.columns = ["name", "lat", "lng", "population"]
df2.head()

In [ ]:
df2.query("name == 'Mandalay'")

In [ ]:
# create a geopandas object of the existing pandas df
geometry = [Point(pos) for pos in zip(df2['lng'], df2['lat'])]
gdf = gpd.GeoDataFrame(df2, geometry=geometry)

In [ ]:
gdf.head()

In [ ]:
gdf = gdf.drop(['lat', 'lng'], axis=1) # remove redundant data

In [ ]:
gdf.plot(figsize=(8, 6));

In [ ]:
# get countries with more than 10 million people living in them
bigcities = gdf[gdf.population > 10_000_000]
bigcities = bigcities.sort_values(['population'], ascending=False)
bigcities

In [ ]:
# load a countries dataset
countries = gpd.read_file("data/vectors/countries/ne_10m_admin_0_countries.shp", encoding="utf-8")
countries.head()

In [ ]:
countries_4cols = countries[["NAME", "POP_EST", "geometry"]]
countries_4cols.head()

In [ ]:
myanmar = countries_4cols[countries_4cols['NAME'] == "Myanmar"]
myanmar

In [ ]:
myanmar.plot(figsize=(15,9));

In [ ]:
restoftheworld = countries_4cols[countries_4cols['NAME'] != "Myanmar"]

ax = myanmar.plot(figsize=(15,9), color="red")
restoftheworld.plot(ax=ax, color="blue");

### looking at cities in myanmar

In [ ]:
myanmarcities = gdf[gdf.within(myanmar.iloc[0].geometry)]
myanmarcities.head()

### list most populated cities in myanmar

In [ ]:
myanmarcities.sort_values('population', ascending=False).head()

### listing airports in myanmar

In [ ]:
# first we read in the data about airports
airports = gpd.read_file("data/vectors/airports/ne_10m_airports.shp", encoding="utf-8")
airports.head()

In [ ]:
myanmarairports = airports[airports.within(myanmar.iloc[0].geometry)]
myanmarairports.head()